## Project: Analysis and Classification of customer churn in telecommunication companies

### Author: Munezero Mihigo

### Date: 13 November 2021

## Project Objective

### The objective of the task is to predict whether the customer will churn or not.

In [3]:
# Import libraries
import numpy as np
import pandas as pd
from scipy.stats.mstats import gmean
import seaborn as sns
import matplotlib.pyplot as plt